### 1️⃣ PySpark'ı Başlatma ve SparkSession Oluşturma
Bu adımda PySpark'ın çalışması için gerekli ortam değişkenlerini ayarlıyor ve bir **Spark oturumu (SparkSession)** oluşturuyoruz.


In [2]:
import os
import findspark
from pyspark.sql import SparkSession

# PySpark ve Java'nın yolunu tanımla
os.environ["SPARK_HOME"] = "c:\\users\\bedirhan\\miniconda3\\envs\\veri_uygulamalari\\lib\\site-packages\\pyspark"
os.environ["JAVA_HOME"] = "C:\\Program Files\\Eclipse Adoptium\\jdk-11.0.26.4-hotspot"  # Java 11 yolunu doğrula

findspark.init()

# Spark oturumunu başlat
spark = SparkSession.builder.appName("CaliforniaHousingRegression").getOrCreate()



### 2️⃣ CSV Dosyasını Okuma ve İlk 5 Satırı Görüntüleme
Bu adımda **housing.csv** dosyasını PySpark DataFrame olarak yüklüyoruz.  
- **`header=True`** → İlk satırı sütun isimleri olarak kullan.  
- **`inferSchema=True`** → Sütunların veri tiplerini otomatik olarak belirle.  


In [3]:
# Veriyi oku
data_path = "housing.csv"  # Dosyanın bulunduğu dizini kontrol et!
df = spark.read.csv(data_path, header=True, inferSchema=True)

# Verinin ilk 5 satırına bakalım
df.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

### 3️⃣ Verinin Şemasını (Schema) İnceleme
Bu adımda verinin şemasını (sütun adları ve veri türlerini) görüntülüyoruz.  
- **Hangi sütunların sayısal (`double`), hangi sütunların kategorik (`string`) olduğunu görebiliriz.**  
- **Eksik değer alabilen (`nullable=True`) sütunları tespit edebiliriz.**


In [4]:
df.printSchema()


root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



### 4️⃣ Eksik Değerleri Sayma
Bu adımda **veri setindeki eksik (boş) değerleri tespit ediyoruz.**  
- **Her sütunda kaç tane eksik (null) değer olduğunu hesaplıyoruz.**  
- **Sonucu tablo halinde ekrana yazdırıyoruz.**  


In [5]:
from pyspark.sql.functions import col, sum

# Eksik değerleri say
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()


+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|        0|       0|                 0|          0|           207|         0|         0|            0|                 0|              0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



### 5️⃣ Kategorik Değerleri (Distinct) İnceleme  
Bu adımda **"ocean_proximity"** sütunundaki **farklı (distinct) değerleri** inceliyoruz.  

- **Bu sütunda tekrar eden değerleri kaldırarak benzersiz kategorileri gösteriyoruz.**  
- **Çıktı, veride kaç farklı kategorik değer olduğunu anlamamızı sağlar.**  
 


In [6]:
df.select("ocean_proximity").distinct().show()


+---------------+
|ocean_proximity|
+---------------+
|         ISLAND|
|     NEAR OCEAN|
|       NEAR BAY|
|      <1H OCEAN|
|         INLAND|
+---------------+



### 6️⃣ Veri Setinin İstatistiksel Özelliklerini İnceleme  
Bu adımda **veri setindeki sayısal sütunların temel istatistiklerini** görüntülüyoruz.  

**Çıktıda bulunan bilgiler:**  
- **`count`** → Her sütundaki veri sayısı.  
- **`mean`** → Ortalama değer.  
- **`stddev`** → Standart sapma (veri yayılımı).  
- **`min` ve `max`** → Minimum ve maksimum değerler.  
 


In [7]:
df.describe().show()


+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|summary|          longitude|         latitude|housing_median_age|       total_rooms|    total_bedrooms|        population|       households|     median_income|median_house_value|ocean_proximity|
+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|  count|              20640|            20640|             20640|             20640|             20433|             20640|            20640|             20640|             20640|          20640|
|   mean|-119.56970445736148| 35.6318614341087|28.639486434108527|2635.7630813953488| 537.8705525375618|1425.4767441860465|499.5396802325581|3.8706710029070246|206855.81690891474|           NULL|
| stddev|  2.0035317

### 7️⃣ Eksik Değerleri Ortalamayla Doldurma  
Bu adımda **"total_bedrooms"** sütunundaki **eksik değerleri sütunun ortalaması ile dolduruyoruz.**  
- Eksik verileri **ortalama değer ile doldurmak**, verinin dağılımını bozmadan tamamlama yöntemidir.  
- **Eksik verilerin etkisini azaltarak modelimizin doğruluğunu artırabiliriz.**  

📌 **Adımlar:**  
1️⃣ **"total_bedrooms"** sütununun ortalamasını hesapla.  
2️⃣ **Eksik değerleri bu ortalama ile doldur.**  
3️⃣ **Eksik veri olup olmadığını tekrar kontrol et.**  


In [8]:
from pyspark.sql.functions import mean

# total_bedrooms sütununun ortalamasını hesapla
bedroom_mean = df.select(mean(col("total_bedrooms"))).collect()[0][0]

# Eksik değerleri ortalama ile doldur
df = df.fillna({"total_bedrooms": bedroom_mean})

# Kontrol edelim
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()


+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|        0|       0|                 0|          0|             0|         0|         0|            0|                 0|              0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



### 8️⃣ Kategorik Değişkeni Sayısal Hale Getirme (`StringIndexer`)  
Bu adımda **"ocean_proximity"** sütunundaki **metin (kategorik) değerleri sayısal değerlere çeviriyoruz.**  
- Makine öğrenmesi modelleri sayısal verilerle çalıştığı için **kategorik sütunları sayısal hale getirmek gereklidir.**  
- **"StringIndexer" en sık görülen kategoriyi 0, diğerlerini sırasıyla 1, 2, 3 olarak etiketler.**  

📌 **Adımlar:**  
1️⃣ **"ocean_proximity" sütununu sayısal hale getir.**  
2️⃣ **Yeni sütunu ("ocean_proximity_index") ekleyerek veri setini dönüştür.**  
3️⃣ **Orijinal ve dönüştürülmüş sütunları göster.**  


In [9]:
from pyspark.ml.feature import StringIndexer

# StringIndexer ile ocean_proximity kategorik sütununu sayısal hale çevir
indexer = StringIndexer(inputCol="ocean_proximity", outputCol="ocean_proximity_index")
df = indexer.fit(df).transform(df)

# Yeni sayısal sütunu göster
df.select("ocean_proximity", "ocean_proximity_index").show(5)


+---------------+---------------------+
|ocean_proximity|ocean_proximity_index|
+---------------+---------------------+
|       NEAR BAY|                  3.0|
|       NEAR BAY|                  3.0|
|       NEAR BAY|                  3.0|
|       NEAR BAY|                  3.0|
|       NEAR BAY|                  3.0|
+---------------+---------------------+
only showing top 5 rows



### 9️⃣ Kategorik Verilerin Dağılımını İnceleme (`groupBy()`)  
Bu adımda **ocean_proximity_index sütunundaki farklı kategorilerin kaç kez tekrarlandığını buluyoruz.**  
 


In [10]:
df.groupBy("ocean_proximity_index").count().orderBy("count", ascending=False).show()


+---------------------+-----+
|ocean_proximity_index|count|
+---------------------+-----+
|                  0.0| 9136|
|                  1.0| 6551|
|                  2.0| 2658|
|                  3.0| 2290|
|                  4.0|    5|
+---------------------+-----+



### 🔟 Kategorik Değerlerin Sayısal Karşılıklarını Görme (`distinct()`)  
Bu adımda **orijinal kategorik değerlerin (ocean_proximity) sayısal karşılıklarını (ocean_proximity_index) inceliyoruz.**  



In [11]:
df.select("ocean_proximity", "ocean_proximity_index").distinct().show()


+---------------+---------------------+
|ocean_proximity|ocean_proximity_index|
+---------------+---------------------+
|         INLAND|                  1.0|
|     NEAR OCEAN|                  2.0|
|       NEAR BAY|                  3.0|
|         ISLAND|                  4.0|
|      <1H OCEAN|                  0.0|
+---------------+---------------------+



###  Bağımsız Değişkenleri Tek Bir Sütunda Birleştirme (`VectorAssembler`)  
Bu adımda, **modelde kullanacağımız tüm bağımsız değişkenleri (`features`) tek bir sütunda birleştiriyoruz.**  

📌 **Adımlar:**  
1️⃣ **Hangi değişkenleri kullanacağımızı belirle (`feature_cols`).**  
2️⃣ **Tüm bağımsız değişkenleri `features` sütununda birleştir (`VectorAssembler`).**  
3️⃣ **Yeni oluşturulan `features` sütununu ve hedef değişkeni (`median_house_value`) göster.**  

📌 **Bu işlem neden önemli?**  
- **Makine öğrenmesi modelleri, özellikleri tek bir sütunda vektör olarak bekler.**  
- **Spark ML için tüm bağımsız değişkenleri tek bir "features" sütununa çevirmeliyiz.**  


In [12]:
from pyspark.ml.feature import VectorAssembler

# Modelde kullanacağımız bağımsız değişkenler (features)
feature_cols = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms",
                "population", "households", "median_income", "ocean_proximity_index"]

# VectorAssembler ile tüm bağımsız değişkenleri tek bir "features" sütununa çevir
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(df)

# Yeni oluşan 'features' sütununu ve hedef değişkeni gösterelim
df.select("features", "median_house_value").show(5, truncate=False)


+-----------------------------------------------------------+------------------+
|features                                                   |median_house_value|
+-----------------------------------------------------------+------------------+
|[-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,3.0]    |452600.0          |
|[-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,3.0]|358500.0          |
|[-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,3.0]   |352100.0          |
|[-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,3.0]   |341300.0          |
|[-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,3.0]   |342200.0          |
+-----------------------------------------------------------+------------------+
only showing top 5 rows



### 1️⃣1️⃣ Veriyi Eğitim ve Test Olarak Bölme  
Bu adımda **veriyi eğitim (%80) ve test (%20) olacak şekilde ikiye ayırıyoruz.**  

 


In [13]:
# Veriyi eğitim (%80) ve test (%20) olarak böl
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Eğitim ve test veri sayısını göster
print("Eğitim veri seti boyutu:", train_data.count())
print("Test veri seti boyutu:", test_data.count())


Eğitim veri seti boyutu: 16560
Test veri seti boyutu: 4080


###  Lineer Regresyon Modelini Eğitme  
Bu adımda **Lineer Regresyon modeli oluşturuyor ve eğitim verisiyle eğitiyoruz.**  

📌 **Bu işlem neden önemli?**  
- **Model bağımsız değişkenlerin fiyat üzerindeki etkisini öğrenir.**  
- **Katsayılar, hangi değişkenlerin fiyatı nasıl etkilediğini gösterir.**  
- **Elde edilen denklem, modelin öğrendiği fiyat tahmin formülüdür.**  


In [14]:
from pyspark.ml.regression import LinearRegression

# Lineer Regresyon Modelini Tanımla
lr = LinearRegression(featuresCol="features", labelCol="median_house_value")

# Modeli eğitim verisi ile eğit
lr_model = lr.fit(train_data)

# Modelin katsayılarını ve sabit terimini yazdır
print("Katsayılar (Coefficients):", lr_model.coefficients)
print("Sabit Terim (Intercept):", lr_model.intercept)


Katsayılar (Coefficients): [-42998.21475945207,-42706.99559442026,1144.740818422602,-5.135908972631532,80.23713743850453,-46.47731286507485,87.57201183285588,39356.34067186512,-1701.0152504894525]
Sabit Terim (Intercept): -3603816.30374622


###  Modeli Test Verisi Üzerinde Çalıştırma ve Tahmin Yapma  
Bu adımda, **eğittiğimiz Lineer Regresyon modelini test verisi üzerinde çalıştırıyoruz** ve **ev fiyatlarını tahmin ediyoruz.**  
 


In [15]:
# Modeli test verisi üzerinde çalıştır ve tahmin yap
predictions = lr_model.transform(test_data)

# İlk 5 tahmini göster
predictions.select("features", "median_house_value", "prediction").show(5)


+--------------------+------------------+------------------+
|            features|median_house_value|        prediction|
+--------------------+------------------+------------------+
|[-124.3,41.84,17....|          103600.0| 100332.6859256993|
|[-124.23,40.54,52...|          106700.0|190939.19476755708|
|[-124.23,41.75,11...|           73200.0| 74378.07399941469|
|[-124.19,40.73,21...|           90100.0|161986.40932086855|
|[-124.18,40.78,34...|           67000.0| 119303.9588295496|
+--------------------+------------------+------------------+
only showing top 5 rows



### 1️⃣4️⃣ Modelin Performansını Değerlendirme (RMSE ve R²)  
Bu adımda, **Lineer Regresyon modelinin tahmin başarısını ölçüyoruz.**  

📌 **Ölçülen metrikler:**  
- **RMSE (Kök Ortalama Kare Hatası)** → Tahminlerin hata miktarını ölçer (Daha düşük RMSE = Daha iyi model).  
- **R² (R-Kare)** → Modelin veri setini ne kadar iyi açıkladığını gösterir (1'e yakınsa model iyi çalışıyor demektir).  

📌 **Adımlar:**  
1️⃣ **RMSE’yi hesapla.**  
2️⃣ **R² (R-Kare) değerini hesapla.**  
3️⃣ **Sonuçları ekrana yazdır.**  


In [16]:
from pyspark.ml.evaluation import RegressionEvaluator

# Modelin performansını değerlendirme
evaluator = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

# R² (R-Kare) metriğini hesapla
r2_evaluator = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="r2")
r2 = r2_evaluator.evaluate(predictions)

# Sonuçları yazdır
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-Kare (R²): {r2}")


Root Mean Squared Error (RMSE): 71796.88026080727
R-Kare (R²): 0.6274325466994999


### 1️⃣5️⃣ StandardScaler ile Özellikleri Ölçeklendirme  
Bu adımda, **bağımsız değişkenleri ölçeklendirerek modelin daha iyi öğrenmesini sağlıyoruz.**  

📌 **Adımlar:**  
1️⃣ **StandardScaler kullanarak tüm bağımsız değişkenleri ölçekle.**  
2️⃣ **Yeni ölçeklenmiş özellikleri ("scaled_features") oluştur.**  
3️⃣ **Sonuçları ekrana yazdır ve ölçeklemenin doğru çalıştığını kontrol et.**  

📌 **Bu işlem neden önemli?**  
- **Bazı değişkenler çok büyük, bazıları çok küçük olabilir.**  
- **Büyük değerli değişkenler modelin diğer değişkenleri göz ardı etmesine neden olabilir.**  
- **Ölçekleme ile tüm değişkenleri aynı seviyeye getirerek modelin daha iyi öğrenmesini sağlıyoruz.**  


In [17]:
from pyspark.ml.feature import StandardScaler

# StandardScaler ile tüm özellikleri ölçekleyelim
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
scaler_model = scaler.fit(df)
df = scaler_model.transform(df)

# Ölçeklenmiş özellikleri görelim
df.select("scaled_features").show(5, truncate=False)


+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|scaled_features                                                                                                                                                                     |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[-1.3278030546902004,1.0525227849496404,0.9821188656747666,-0.804799599801809,-0.9752042257163941,-0.9744049915469923,-0.977009185045236,2.3447089561176147,2.0817600895927892]     |
|[-1.3228118684350991,1.0431592803959744,-0.6070042082805048,2.0458405373571247,1.3550553701672787,0.8614179998296625,1.6699205725917976,2.3321814648403056,2.0817600895927892]      |
|[-1.3327942409452944,1.0384775281191432,1.8561365563501657,-0.5357329073251578,-0.82

In [21]:
df.printSchema()


root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_index: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)



In [22]:
train_data.printSchema()
test_data.printSchema()


root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_index: double (nullable = false)
 |-- features: vector (nullable = true)

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 

In [23]:
# Model eğitiminden önce veriyi tekrar bölelim (Ölçeklenmiş veriye göre)
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Bölünmüş veri kümelerini kontrol et
train_data.printSchema()
test_data.printSchema()


root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_index: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 

### 1️⃣6️⃣ Random Forest Modeli ile Tahmin Yapma  
Bu adımda, **Random Forest algoritması kullanarak ev fiyatlarını tahmin ediyoruz.**  

📌 **Adımlar:**  
1️⃣ **Random Forest modelini oluştur (`RandomForestRegressor`).**  
2️⃣ **Modeli eğitim verisi ile eğit (`fit`).**  
3️⃣ **Test verisi üzerinde tahmin yap (`transform`).**  
4️⃣ **Tahmin edilen fiyatları gerçek fiyatlarla karşılaştır (`show`).**  
 


In [24]:
from pyspark.ml.regression import RandomForestRegressor

# Random Forest modelini scaled_features ile tanımla
rf = RandomForestRegressor(featuresCol="scaled_features", labelCol="median_house_value", numTrees=50)

# Modeli eğit
rf_model = rf.fit(train_data)

# Modeli test verisi üzerinde çalıştır
predictions_rf = rf_model.transform(test_data)

# İlk 5 tahmini görelim
predictions_rf.select("scaled_features", "median_house_value", "prediction").show(5, truncate=False)


+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+------------------+
|scaled_features                                                                                                                                                                     |median_house_value|prediction        |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+------------------+
|[-2.3609786094955965,2.9064966865751956,-0.9248288230715591,0.018902012430801425,-0.01638707367418689,-0.1602497255300247,-0.11387991625055117,-0.441815668795915,1.08628877737642] |103600.0          |138007.8191091182 |
|[-2.3260403057099097,2.2978688905870066,1.8561365563501657,0.02669440386538928,-0.20242622257789875,-0.241488645783

### 1️⃣7️⃣ Random Forest Modelinin Performansını Değerlendirme  
Bu adımda, **Random Forest modelinin tahmin başarısını ölçüyoruz.**  

📌 **Ölçülen metrikler:**  
- **RMSE (Kök Ortalama Kare Hatası)** → Modelin tahminlerinin hata miktarını ölçer.  
- **R² (R-Kare)** → Modelin veri setini ne kadar iyi açıkladığını gösterir (1'e yakınsa model iyi çalışıyor demektir). 

In [25]:
from pyspark.ml.evaluation import RegressionEvaluator

# RMSE (Hata Metrikleri)
evaluator_rmse = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="rmse")
rmse_rf = evaluator_rmse.evaluate(predictions_rf)

# R² (Modelin ne kadar iyi öğrendiğini gösteren metrik)
evaluator_r2 = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="r2")
r2_rf = evaluator_r2.evaluate(predictions_rf)

# Sonuçları yazdır
print(f"Random Forest RMSE: {rmse_rf}")
print(f"Random Forest R-Kare (R²): {r2_rf}")


Random Forest RMSE: 71397.04195332128
Random Forest R-Kare (R²): 0.6315706635644186


### 1️⃣8️⃣ Yeni Öznitelikler Ekleyerek Modeli Güçlendirme  
Bu adımda, **veri setine yeni öznitelikler ekleyerek modelin daha iyi tahmin yapmasını sağlıyoruz.**  

📌 **Eklenen öznitelikler:**  
1️⃣ **"rooms_per_person"** → Kişi başına düşen oda sayısı.  
2️⃣ **"rooms_per_household"** → Hane başına düşen oda sayısı.  
3️⃣ **"income_per_household"** → Hane başına düşen gelir.  

📌 **Bu işlem neden önemli?**  
- **Daha fazla bilgi içeren bir veri seti, modelin daha doğru tahmin yapmasını sağlar.**  
- **Özellikle gelir ve oda sayısı gibi faktörler, konut fiyatlarını belirlemede önemli olabilir.**  


In [26]:
from pyspark.sql.functions import col

# Yeni öznitelikleri hesapla
df = df.withColumn("rooms_per_person", col("total_rooms") / col("population"))
df = df.withColumn("rooms_per_household", col("total_rooms") / col("households"))
df = df.withColumn("income_per_household", col("median_income") / col("households"))

# Yeni eklenen öznitelikleri görelim
df.select("rooms_per_person", "rooms_per_household", "income_per_household").show(5)


+------------------+-------------------+--------------------+
|  rooms_per_person|rooms_per_household|income_per_household|
+------------------+-------------------+--------------------+
| 2.732919254658385|  6.984126984126984| 0.06607301587301588|
|2.9566847147022073|  6.238137082601054|0.007294727592267135|
|2.9576612903225805|  8.288135593220339| 0.04100225988700565|
| 2.283154121863799| 5.8173515981735155|0.025767579908675797|
| 2.879646017699115|  6.281853281853282| 0.01485019305019305|
+------------------+-------------------+--------------------+
only showing top 5 rows



### 1️⃣9️⃣ Yeni Özniteliklerle "features" Vektörünü Güncelleme  
Bu adımda, **veri setindeki eski "features" sütununu kaldırıyor ve yeni eklenen değişkenlerle yeniden oluşturuyoruz.**  
📌 **Adımlar:**  
1️⃣ **Eski "features" sütununu kaldır.**  
2️⃣ **Yeni bağımsız değişkenler listesini oluştur.**  
3️⃣ **Tüm değişkenleri birleştirerek yeni "features" sütununu oluştur.**  
4️⃣ **Yeni "features" sütununun ilk 5 satırını göster.**  
 


In [28]:
# Önce eski 'features' sütununu düşür
df = df.drop("features")

from pyspark.ml.feature import VectorAssembler

# Güncellenmiş bağımsız değişkenler listesi (yeni eklenen özniteliklerle)
feature_cols = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms",
                "population", "households", "median_income", "ocean_proximity_index",
                "rooms_per_person", "rooms_per_household", "income_per_household"]

# Yeni features vektörünü oluştur
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(df)

# Yeni features sütununu görelim
df.select("features").show(5, truncate=False)


+---------------------------------------------------------------------------------------------------------------------+
|features                                                                                                             |
+---------------------------------------------------------------------------------------------------------------------+
|[-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,3.0,2.732919254658385,6.984126984126984,0.06607301587301588]      |
|[-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,3.0,2.9566847147022073,6.238137082601054,0.007294727592267135]|
|[-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,3.0,2.9576612903225805,8.288135593220339,0.04100225988700565]    |
|[-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,3.0,2.283154121863799,5.8173515981735155,0.025767579908675797]   |
|[-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,3.0,2.879646017699115,6.281853281853282,0.01485019305019305]     |
+---------------------------------------

###   StandardScaler ile Özellikleri Ölçeklendirme  
Bu adımda, **bağımsız değişkenleri ölçeklendirerek modelin daha iyi öğrenmesini sağlıyoruz.**  

📌 **Adımlar:**  
1️⃣ **StandardScaler kullanarak tüm bağımsız değişkenleri ölçekle.**  
2️⃣ **Yeni ölçeklenmiş özellikleri ("scaled_features") oluştur.**  
3️⃣ **Sonuçları ekrana yazdır ve ölçeklemenin doğru çalıştığını kontrol et.**  
 

In [30]:
# Önce eski 'scaled_features' sütununu kaldır
df = df.drop("scaled_features")

from pyspark.ml.feature import StandardScaler

# StandardScaler ile ölçekleme yap
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
scaler_model = scaler.fit(df)
df = scaler_model.transform(df)

# Yeni ölçeklenmiş özellikleri kontrol edelim
df.select("scaled_features").show(5, truncate=False)


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|scaled_features                                                                                                                                                                                                                                |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[-1.3278030546902004,1.0525227849496404,0.9821188656747666,-0.804799599801809,-0.9752042257163941,-0.9744049915469923,-0.977009185045236,2.3447089561176147,2.0817600895927892,0.6596305965114945,0.6285442264151613,0.420730403000244]        |
|[-1.3228118684350991,1.04315928

In [31]:
# Model eğitiminden önce veriyi tekrar bölelim (Ölçeklenmiş yeni veriye göre)
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Bölünmüş veri kümelerini kontrol edelim
train_data.printSchema()
test_data.printSchema()


root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_index: double (nullable = false)
 |-- rooms_per_person: double (nullable = true)
 |-- rooms_per_household: double (nullable = true)
 |-- income_per_household: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (null

### 2️⃣0️⃣ Random Forest Modelini Yeniden Eğitme ve Tahmin Yapma  
Bu adımda, **Random Forest modelini yeni eklenen özniteliklerle tekrar eğitiyoruz.**  

📌 **Adımlar:**  
1️⃣ **Random Forest modelini oluştur (`RandomForestRegressor`).**  
2️⃣ **Modeli eğitim verisi ile eğit (`fit`).**  
3️⃣ **Test verisi üzerinde tahmin yap (`transform`).**  
4️⃣ **Tahmin edilen fiyatları gerçek fiyatlarla karşılaştır (`show`).**  

📌 **Bu işlem neden önemli?**  
- **Yeni eklenen özelliklerle modelin tahmin gücünü artırıyoruz.**  
- **Random Forest, çok sayıda karar ağacı kullanarak daha güçlü tahminler yapar.**  



In [32]:
from pyspark.ml.regression import RandomForestRegressor

# Random Forest modelini scaled_features ile tanımla
rf = RandomForestRegressor(featuresCol="scaled_features", labelCol="median_house_value", numTrees=50)

# Modeli eğit
rf_model = rf.fit(train_data)

# Modeli test verisi üzerinde çalıştır
predictions_rf = rf_model.transform(test_data)

# İlk 5 tahmini görelim
predictions_rf.select("scaled_features", "median_house_value", "prediction").show(5, truncate=False)


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+------------------+
|scaled_features                                                                                                                                                                                                                                   |median_house_value|prediction        |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+------------------+
|[-2.3609786094955965,2.9064966865751956,-0.9248288230715591,0.018902012430801425,-0.01638707367418689,-0.1602497255300247,-0.11387991625055117,-0.4418

In [33]:
from pyspark.ml.evaluation import RegressionEvaluator

# RMSE (Hata Metrikleri)
evaluator_rmse = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="rmse")
rmse_rf = evaluator_rmse.evaluate(predictions_rf)

# R² (Modelin ne kadar iyi öğrendiğini gösteren metrik)
evaluator_r2 = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="r2")
r2_rf = evaluator_r2.evaluate(predictions_rf)

# Sonuçları yazdır
print(f"Random Forest (Yeni Özellikler) RMSE: {rmse_rf}")
print(f"Random Forest (Yeni Özellikler) R-Kare (R²): {r2_rf}")


Random Forest (Yeni Özellikler) RMSE: 67864.08924408912
Random Forest (Yeni Özellikler) R-Kare (R²): 0.667130643626022


### 2️⃣1️⃣ XGBoost Modeli ile Ev Fiyatlarını Tahmin Etme  
Bu adımda, **XGBoost algoritmasını kullanarak konut fiyatlarını tahmin ediyoruz.**  

📌 **Neden XGBoost kullanıyoruz?**  
- **XGBoost, karar ağaçlarından daha güçlü bir makine öğrenmesi modelidir.**  
- **Özellikle büyük veri setlerinde ve karmaşık ilişkilerde çok başarılıdır.**  
- **Random Forest yerine XGBoost kullanarak modelin doğruluğunu artırmayı hedefliyoruz.**  

  


In [34]:
from xgboost import XGBRegressor
import numpy as np

# Pandas ve NumPy kullanarak veriyi XGBoost için uygun hale getirelim
train_pd = train_data.select("scaled_features", "median_house_value").toPandas()
test_pd = test_data.select("scaled_features", "median_house_value").toPandas()

# Feature ve Label ayrımı
X_train = np.array(train_pd["scaled_features"].tolist())  # Bağımsız değişkenler (features)
y_train = np.array(train_pd["median_house_value"])        # Bağımlı değişken (label)

X_test = np.array(test_pd["scaled_features"].tolist())    # Test özellikleri
y_test = np.array(test_pd["median_house_value"])          # Test gerçek değerleri

# XGBoost modelini tanımla
xgb_model = XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42)

# Modeli eğit
xgb_model.fit(X_train, y_train)

# Model ile tahmin yap
y_pred = xgb_model.predict(X_test)

# İlk 5 tahmini göster
for i in range(5):
    print(f"Gerçek: {y_test[i]} Tahmin: {y_pred[i]}")


Gerçek: 103600.0 Tahmin: 119472.546875
Gerçek: 106700.0 Tahmin: 91329.21875
Gerçek: 73200.0 Tahmin: 110369.7421875
Gerçek: 90100.0 Tahmin: 114152.0234375
Gerçek: 67000.0 Tahmin: 75674.1171875


### 2️⃣2️⃣ XGBoost Modelinin Performansını Değerlendirme  
Bu adımda, **XGBoost modelinin tahmin başarısını ölçüyoruz.**  

📌 **Ölçülen metrikler:**  
- **RMSE (Kök Ortalama Kare Hatası)** → Modelin tahminlerinin hata miktarını ölçer.  
- **R² (R-Kare)** → Modelin veri setini ne kadar iyi açıkladığını gösterir (1'e yakınsa model iyi çalışıyor demektir).  




In [35]:
from sklearn.metrics import mean_squared_error, r2_score

# RMSE'yi hesapla
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred))

# R² (Modelin doğruluk oranı)
r2_xgb = r2_score(y_test, y_pred)

# Sonuçları yazdır
print(f"XGBoost RMSE: {rmse_xgb}")
print(f"XGBoost R-Kare (R²): {r2_xgb}")


XGBoost RMSE: 46296.1434138838
XGBoost R-Kare (R²): 0.8450885940863098


### 📌 Konut Fiyatlarının Tahmini - Proje Özeti  

Bu projede, **California Housing Prices** veri seti kullanılarak konut fiyatlarının tahmini için farklı makine öğrenmesi modelleri uygulanmıştır.  

#### 📌 Adımlar:  
1️⃣ **Veri yükleme ve ön işleme**  
   - Eksik veriler dolduruldu.  
   - Kategorik veriler sayısal hale getirildi.  

2️⃣ **Yeni öznitelikler oluşturma**  
   - Kişi başına düşen oda sayısı, hane başına düşen gelir gibi ek özellikler eklendi.  

3️⃣ **Özellik mühendisliği ve ölçekleme**  
   - Veriler `VectorAssembler` ile birleştirildi.  
   - `StandardScaler` ile ölçeklendirildi.  

4️⃣ **Farklı modellerin eğitilmesi**  
   - **Lineer Regresyon**  
   - **Random Forest**  
   - **XGBoost**  

5️⃣ **Model performanslarının karşılaştırılması**  
   - RMSE ve R² metrikleri hesaplandı.  
   - **En iyi sonuç XGBoost modeli ile alındı (RMSE: ~46,296, R²: ~0.845).**  

#### 📌 Sonuç:  
✅ **XGBoost modeli, konut fiyatlarını en doğru şekilde tahmin etti.**  

